**Preliminary note:** If you don't feel very comfortable with programming in general or with the Python language in particular, it is recommended to learn the basis of the language by following this [interactive Python tutorial](https://www.codecademy.com/learn/python). It is quite well explained and you can practice by typing and executing pieces of code in the webpage.

In [1]:
from simulator_interface import get_session
simulator, epuck = get_session()

**Reminder :** Each time you encounter a cell in this notebook where there is code (as in the cells just above), you have to execute the cell by clicking on it and pressing `Shift+Enter` (unless it is explicitly specified not to do it). If at one point things are not going as expected (e.g. the robot doesn't move the way it should in the simulator, do the following steps:
- Stop any code that can still be executing by pressing the "stop-like" button in the top menu bar of this document. 
- Go to the V-REP simulator and stop it by pressing the "stop-like" button in it (you might need to increase the size of the window to see it).
- Restart the notebook by clicking `Kernel -> Restart` in the menu (sometimes doing it twice might help).
- Execute the cell above.

If it still doesn't work, quit the simulator (don't save the scene, or in another name). Then re-open V-REP and load the scene called `epuck-scene-1.ttt` in `Documents/rti2016/pyvrep_epuck/vrep_scenes`.

But hopefully it should not happen too often. Tell me otherwise.

During the last practical session we have seen how to set the robot left and wheel speed as well as how to read the values returned by the left and right proximiter. We have programmed a first simple behavior where the robot slows down when approaching an obstacle. 
Here is a possible solution for this behavior:

In [2]:
# Repeat 1000 times the indented code:
for i in range(1000):
    # Read the proximeter values and store them in the "left" and "right" variables
    left, right = epuck.prox_activations()
    
    # Compute the sum of the values returned by the left and right proximeters.
    # This sum will be between 0 and 2 because both "left" and "right" are between 0 and 1
    sum_of_proxs = left + right
    
    # Compute the speed that will be applied to both wheels. 
    # The closer the obstacle (i.e. the higher the value returned by the proximeters). the lower the speed should be.
    wheel_speed = 2 - sum_of_proxs
    
    # Set the speed of both wheel to the value we just have computed
    epuck.left_spd = wheel_speed
    epuck.right_spd = wheel_speed
    
    
    # Waits for 100 milliseconds before starting the next iteration (to avoid overloading you computer)
    epuck.wait(0.1)

KeyboardInterrupt: 

This example behavior defined above illustrates the general structure of a behavior. 

**Definition: ** a behavior consists in a loop repeated at a certain frequency where (1) the values of relevant sensors is read, (2) some computation is performed using these values and (3) commands are sent to the robot motors according to the result of this computation.

In the example above, the frequency of the behavior is approximately 10 Hz, i.e. the core of the loop is executed approximately 10 times per second (because we wait for 0.1s at the end of each iteration. This is an approximation because we don't take into account the time needed to execute the instructions occuring before the waiting period. Step (1) corresponds to the reading of the left and right proximeter activations. Step (2) corresponds to the computation of `wheel_speed` according to the some of the proximeter activations. Finally, Step (3) corresponds to setting the speed of both wheels to the value of `wheel_speed`.

Note that the code above will take a while to be executed (approximately `1000 * 0.1 = 100` seconds). During this time, you can't execute anything else in this notebook. To stop the execution before it terminates by itself, press the "stop-like" button in the top menu bar of this document. 

This approach has two major drawbacks:
- Only one behavior can run at a time.
- The behavior has a fixed duration (at one point it will stop)
- One can't stop a behavior programatically (instead one has to press the "stop-like" button).

To overcome this problem, we provide a more flexible method for defining and executing behaviors. Let's rewrite the behavior above using that method. First make sure the previous code is not still being executed by pressing the "stop-like" button in the top menu bar of this document. Now, defining a behavior boils down to define a function which includes the core of the behavioral loop:

** TO CHECK: behavior definition could return wheel speed (as below). Better for mixing behaviors.**

In [2]:
def slow_down(epuck):
    # read the sensor values
    left, right = epuck.prox_activations()
    
    # do some computation
    sum_of_proxs = left + right
    wheel_speed = 2 - sum_of_proxs
    
    # set the motor values
    # epuck.left_spd = wheel_speed
    # epuck.right_spd = wheel_speed 
    return wheel_speed, wheel_speed

In [3]:
def obstacle_avoidance(epuck):
    left, right = epuck.prox_activations()
    return left, right

In [4]:
def behavior_combination(epuck):
    left1, right1 = slow_down(epuck)
    left2, right2 = obstacle_avoidance(epuck)
    return left1 + left2, right1 + right2

In [22]:
import inspect
args = inspect.getargspec(behavior_combination)
args.args

['epuck']

The cell above defines a function called `slow_down` (one can choose an arbitrary name but it's better if it reflects the purpose of the behavior defined in it). In Python, the definition of a function starts with the keyword `def`, followed by the arbitrary name we choose for the function (here `slow_down`), then the arguments of the function in parenthesis (in our case it will be the variable representing the robot, called `epuck`) and finally the symbol `:`. Below, you find the instructions that this function will execute when it will be called. Those instructions need to be intended right with respect to the first line. In this example, the instructions are the exact same than in core of the previous `for` loop, except that we omit the last line (`epuck.wait(0.1)`).

Once the behavior is defined in a function, we can attach it to the robot by executing:


In [5]:
epuck.attach_behavior(behavior_combination, freq=10)

The line above means: attach the behavior defined in the function `slow_down` to the `epuck` robot and set the frequency at which it will be repeated to 10Hz. Note that this instruction does not execute the behavior on the robot, it only informs the robot that the behavior is available to it. In order to run the behavior, we have to execute the following instruction:

In [6]:
epuck.start_behavior(behavior_combination)

The line above means: start running the previously attached behavior `slow_down` on the `epuck`. Executing the above line will basically do the same thing as executing the `for` loop at the start of this document. Here, the function `slow_down` will be executed at a frequency of 10Hz (i.e. 10 times per second) indefinitely. Look in the simulator, you should see the robot slowing down as it approaches an obstacle.

Using this method has the following advantages over the previous method using the `for` loop:
- It is more compact to write and it will allow to better structure your code when you will have to deal with multiple behaviors and multiple robots.
- It better manages the frequency at which the behavior is run. Now the behavior runs at *exactly* 10Hz, whereas in the previous method one can only approximate the true frequency.
- It is not blocking as the previous method was. This means that you can still use this notebook while the behavior is running on the robot. For example, let's read the proximeter activations while the robot is executing the `slow_down` behavior:

In [15]:
epuck.prox_activations()

array([ 0.97370812,  0.9744308 ])

Each time you execute the cell above, you should see the proximeter activation changing because the robot is moving. However, you should avoid setting motor values while a behavior is running since this could conflict with the behavior also setting those values. If you want to stop the behavior, you can execute:

When a behavior is started, it runs indefinitely until you explicitly tell it to stop. To do so, you have to execute:

In [16]:
epuck.stop_behavior(slow_down)

Note that the robot will continue moving using the last wheel speeds that were set by the behavior. You can set both wheel speeds to 0 by executing:

In [17]:
epuck.stop()

Finally, if you don't want the behavior to be attached to the robot anymore (for example if you want to change it), you can execute:

In [17]:
epuck.detach_behavior(slow_down)

At anytime, you can check what behaviors are attached to the robot with:

In [5]:
epuck.check_behaviors()

Behavior " slow_down " is attached and   STARTED.
Behavior "slow_down" is attached and STARTED


**Q1:** To make sure you correctly understand how to attach, start, stop and detach a behavior, complete the following code:

In [ ]:
# First we make sure that no behavior is attach to the robot:
epuck.detach_all_behaviors()

# When checking, it should print "No behavior attached":
epuck.check_behaviors() # This will print "No behavior attached"

# Write just below this line the code that attaches the slow_down behavior

epuck.check_behaviors() # This will print "Behavior "slow_down" is attached and NOT STARTED"

# Write just below this line the code that starts the slow_down behavior

epuck.check_behaviors() # This will print "Behavior "slow_down" is attached and STARTED"

# Write just below this line the code that stops the slow_down behavior

epuck.check_behaviors() # This will print "Behavior "slow_down" is attached and NOT STARTED"

# Write just below this line the code that detaches the slow_down behavior

epuck.check_behaviors() # This will print "No behavior attached"

Let's summarize the method we have just describe to define, attach, start, stop and detach a behavior:

In [2]:
# First, detach all the behaviors that might still be attached to the robot (it is a good practice to do it each time you want to define a new behavior):
epuck.start_all_behaviors()

# Define a behavior where the robot progressively slows down when it approaches an obstacle:
def slow_down(epuck):
    # read the sensor values
    left, right = epuck.prox_activations()
    
    # do some computation
    sum_of_proxs = left + right
    wheel_speed = 2 - sum_of_proxs
    
    # set the motor values
    # epuck.left_spd = wheel_speed
    # epuck.right_spd = wheel_speed 
    return wheel_speed, wheel_speed


# Attach this behavior to the robot, specifying the frequency (in Hz) at which it has to be executed
epuck.attach_behavior(slow_down, freq=10)

In [3]:
# Start the behavior in the simulator
epuck.start_behavior(slow_down)

When executing the code above, you should the behavior being executed on the robot in the simulator. Then, to stop and detach the behavior:

In [4]:
epuck.stop_behavior(slow_down)
epuck.detach_behavior(slow_down)

An alternative way is to stop and detach all the behaviors running on the robot. This avoids having to specify the name of the behavior (`slown_down` in the cell above) and also stops systematically the behavior before detaching it:

In [25]:
epuck.detach_all_behaviors()

Let's now practice a bit.

Remember the example of a car having to stop we have seen in class in [this slide](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame3456). We will realize a sort of crash-test for the E-Puck and determine empirically the maximum reaction time allowed in the "slow_down" behavior. 

**Q1:** Redefine the "slow_down" behavior we have seen above so that the robot move 10 times faster. You can copy-paste the definition of the "slow_down" behavior provided above. Remember to stop and detach the behavior each time you modify its definition.

In [14]:
def heaviside(x):
    if x >= 0:
        return 1.
    else:
        return 0.

In [27]:
# Define the behavior just below
def emergency_break(epuck):
    left, right = epuck.prox_activations()
    
    sum_of_proxs = left + right
    
    wheel_speed = 30. * heaviside(1.5 - sum_of_proxs)
    
    return wheel_speed, wheel_speed

In [7]:
epuck.left_spd

1.3523094822084483

In [28]:
# Then attach and start it 
epuck.attach_behavior(slow_down, freq=10)
epuck.start_behavior(slow_down)
print epuck.left_spd, epuck.right_spd

0.0 0.0


In [8]:
# When you modify the definition above, first execute this cell before re-executing the one above
epuck.detach_all_behaviors()

Behavior obstacle_avoidance detached


In [9]:
# You might also want to immobiize the E-Puck
epuck.stop()
# Then move the robot back to its starting point in by dragging it in V-REP. 
# Alternatively you can stop the simulation in V-REP (easier)

In [34]:
from numpy import exp
from numpy.random import rand
def obstacle_avoidance(epuck):
    left, right = 1 + 3 * epuck.prox_activations() + (rand(2) - 0.5) * 2
    return left, right

In [27]:
rand(2)

array([ 0.10887457,  0.89367336])

In [35]:
epuck.attach_behavior(obstacle_avoidance, freq=10)
epuck.start_all_behaviors()

Behavior obstacle_avoidance started


In [36]:
# When you modify the definition above, first execute this cell before re-executing the one above
epuck.detach_all_behaviors()
epuck.stop()

Behavior obstacle_avoidance detached


In [1]:
from simulator_interface import get_session
simulator, epuck = get_session()

[0, 1, 2, 3, 4, 5, 6, 7]


In [19]:
epuck.proximeters(), epuck.min_index()

(array([ 1208.52081599,  1418.62856531]), 0)

In [2]:
while True:
    epuck.fwd_spd = 0.2
    idx = epuck.min_index()
    rot = (idx - 0.5) #/ 2.
    epuck.rot_spd = rot

KeyboardInterrupt: 

In [17]:
from numpy import array
array([-1, 1]) > 0

array([False,  True], dtype=bool)

In [ ]:
del simulator

In [ ]:
del epuck

In [11]:
epuck.left_spd = 18
epuck.right_spd = 6

**Q2:** Real-time constraint.

In [13]:
epuck.prox_activations("Cup")

array([ 0.        ,  0.66596079])

In [7]:
epuck.proximeters()

array([ 1397.80948896,  1840.01990775])

In [5]:
epuck.no_detection_value

2000.0

In [21]:
epuck.stop()

In [4]:
len(epuck._behaviors)

0